In [1]:
import pandas as pd
dataset = pd.read_json("../Datasets/dataset_humor_train.json", lines=True)
#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset.klass.value_counts())
# Extracción de los textos en arreglos de numpy
X_train = dataset['text'].to_numpy()
# Extracción de las etiquetas o clases de entrenamiento
Y_train = dataset['klass'].to_numpy()

Total de ejemplos de entrenamiento
klass
0    6588
1    3812
Name: count, dtype: int64


In [2]:
import pandas as pd
dataset = pd.read_json("../Datasets/dataset_humor_test.json", lines=True)
#conteo de clases
print("Total de ejemplos de prueba")
print(dataset.klass.value_counts())
# Extracción de los textos en arreglos de numpy
X_test = dataset['text'].to_numpy()
# Extracción de las etiquetas o clases de entrenamiento
Y_test = dataset['klass'].to_numpy()

Total de ejemplos de prueba
klass
-1    5600
Name: count, dtype: int64


In [3]:
from sklearn.model_selection import train_test_split

# Divide el conjunto de entrenamiento en:  entrenamiento (90%) y validación (10%)
X_train, X_val, Y_train, Y_val =  train_test_split(X_train, Y_train, test_size=0.1, stratify=Y_train, random_state=42)

In [4]:
import re

def eliminar_emojis(texto):
    # Expresión regular que cubre la mayoría de emojis y pictogramas
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticonos
        "\U0001F300-\U0001F5FF"  # símbolos y pictogramas
        "\U0001F680-\U0001F6FF"  # transporte y mapas
        "\U0001F1E0-\U0001F1FF"  # banderas
        "\U00002702-\U000027B0"  # otros símbolos
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', texto)

In [5]:
import unicodedata
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer

_STOPWORDS = stopwords.words("spanish")  # agregar más palabras a esta lista si es necesario
stemmer = SnowballStemmer("spanish")
PUNCTUACTION = ";:,.\\-\"'/"
SYMBOLS = "()[]¿?¡!{}~<>|"
NUMBERS= "0123456789"
SKIP_SYMBOLS = set(PUNCTUACTION + SYMBOLS)
SKIP_SYMBOLS_AND_SPACES = set(PUNCTUACTION + SYMBOLS + '\t\n\r ')

def normaliza_texto(input_str,
                    punct=False,
                    accents=False,
                    num=False,
                    max_dup=2):
    """
        punct=False (elimina la puntuación, True deja intacta la puntuación)
        accents=False (elimina los acentos, True deja intactos los acentos)
        num= False (elimina los números, True deja intactos los acentos)
        max_dup=2 (número máximo de símbolos duplicados de forma consecutiva, rrrrr => rr)
    """
    
    nfkd_f = unicodedata.normalize('NFKD', input_str)
    n_str = []
    c_prev = ''
    cc_prev = 0
    for c in nfkd_f:
        if not num:
            if c in NUMBERS:
                continue
        if not punct:
            if c in SKIP_SYMBOLS:
                continue
        if not accents and unicodedata.combining(c):
            continue
        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        n_str.append(c)
        c_prev = c
    texto = unicodedata.normalize('NFKD', "".join(n_str))
    texto = re.sub(r'(\s)+', r' ', texto.strip(), flags=re.IGNORECASE)
    return texto

def mi_preprocesamiento_factory(modo):
    """
    Devuelve una función de preprocesamiento y tokenización según el modo:
    - 'normalizacion'
    - 'normalizacion_stopwords'
    - 'normalizacion_stopwords_stem'
    """

    def preprocesamiento(texto):
        texto = texto.lower()
        #texto = eliminar_emojis(texto)
        texto = re.sub(r"http\S+|www\S+|https\S+", "", texto)
        texto = re.sub(r"@\w+", "", texto)
        texto = normaliza_texto(texto, punct=True)
        return texto

    def tokenizador(texto):
        tokens = word_tokenize(texto)
        if modo in ['normalizacion_stopwords', 'normalizacion_stopwords_stem']:
            tokens = [t for t in tokens if t not in _STOPWORDS and len(t) > 2]
        if modo == 'normalizacion_stopwords_stem':
            tokens = [stemmer.stem(t) for t in tokens]
        return tokens

    return preprocesamiento, tokenizador

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

class Vec_TFID:
    def __init__(self, modo_preproc='normalizacion'):
        self.vec_tfidf = None
        self.modo_preproc = modo_preproc

    def create_matriz_TFID(self, X_train, ngram_config, max_config):
        preproc, token = mi_preprocesamiento_factory(self.modo_preproc)
        self.vec_tfidf = TfidfVectorizer(
            min_df=5,
            analyzer="word",
            preprocessor=preproc,
            tokenizer=token,
            ngram_range=ngram_config,
            max_features=max_config
        )
        X_tfidf = self.vec_tfidf.fit_transform(X_train)
        return X_tfidf.toarray()

    def tranform_matriz_TFID(self, X_test):
        X_tfid = self.vec_tfidf.transform(X_test)
        return X_tfid.toarray()

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
class Vec_Count:
    def __init__(self, modo_preproc='normalizacion'):
        self.vec = None
        self.modo_preproc = modo_preproc

    def create_matriz_Count(self, X_train, ngram_config, max_config):
        preproc, token = mi_preprocesamiento_factory(self.modo_preproc)
        self.vec = CountVectorizer(
            analyzer="word",
            preprocessor=preproc,
            tokenizer=token,
            ngram_range=ngram_config,
            max_features=max_config
        )
        X_vec = self.vec.fit_transform(X_train)
        return X_vec.toarray().astype(np.float32)

    def transform_matriz_Count(self, X_test):
        X_vec = self.vec.transform(X_test)
        return X_vec.toarray().astype(np.float32)


In [60]:
import torch
USE_TFIDF = True      # True: TF-IDF, False: Count
NGRAM_RANGE = (1, 2)
MAX_FEATURES = 15000  # features for vectorizer 
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
preproc = "normalizacion" 
FACTOR_BALANCE = 2
#preproc = "normalizacion_stopwords" 
#preproc = "normalizacion_stopwords_stem"

In [61]:
if USE_TFIDF:
    vectorizer = Vec_TFID(modo_preproc=preproc)

    X_tr = vectorizer.create_matriz_TFID(X_train, 
                                            ngram_config=NGRAM_RANGE, 
                                            max_config=MAX_FEATURES)
    
    X_val_vectorized = vectorizer.tranform_matriz_TFID(X_val)
    X_t = vectorizer.tranform_matriz_TFID(X_test) 
else:
    vectorizer = Vec_Count(modo_preproc=preproc)
    X_tr = vectorizer.create_matriz_Count(X_train, 
                                            ngram_config=NGRAM_RANGE, 
                                            max_config=MAX_FEATURES)
    
    X_val_vectorized = vectorizer.transform_matriz_Count(X_val)
    X_t = vectorizer.transform_matriz_Count(X_test)

d:\Apps\envs\RNA\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# import torch
# import torch.nn as nn
# NUM_CLASSES = 2
# Y_train_one_hot = nn.functional.one_hot(torch.from_numpy(Y_train), num_classes=NUM_CLASSES).float()
# Y_val_one_hot = nn.functional.one_hot(torch.from_numpy(Y_val), num_classes=NUM_CLASSES).float()


In [10]:
from torch.utils.data import DataLoader, TensorDataset
def create_minibatches(X, Y, batch_size):
    # Recibe los documentos en X y las etiquetas en Y
    dataset = TensorDataset(X, Y) # Cargar los datos en un dataset de tensores
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # loader = DataLoader(dataset, batch_size=batch_size)
    return loader

In [ ]:
X_train.shape, Y_train.shape

((9360,), (9360,))

In [ ]:
# Y_train_one_hot.shape,  Y_val_one_hot.shape

In [ ]:
import torch.optim as optim
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        # Definición de capas, funciones de activación e inicialización de pesos
        input_size_h1 = 1024
        input_size_h2 = 256
        input_size_h3 = 64
        self.fc1 = nn.Linear(input_size, input_size_h1)
        self.bn1 = nn.BatchNorm1d(input_size_h1)
        
        self.act1= nn.LeakyReLU(0.1)
        self.drop1 = nn.Dropout(p=0.6)

        self.fc2 = nn.Linear(input_size_h1, input_size_h2)
        self.bn2 = nn.BatchNorm1d(input_size_h2)
        
        self.act2= nn.LeakyReLU(0.1)
        self.drop2 = nn.Dropout(p=0.4)

        self.fc3 = nn.Linear(input_size_h2, input_size_h3)
        self.bn3 = nn.BatchNorm1d(input_size_h3)
        self.act3 = nn.LeakyReLU(0.1)
        self.drop3 = nn.Dropout(p=0.2)

        self.output = nn.Linear(input_size_h3, output_size)
        
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
       
    def forward(self, X):
        # Definición del orden de conexión de las capas y aplición de las funciones de activación
        # x = self.fc1(X)
        # x = self.bn1(x)
        # x = self.act1(x)
        # x = self.drop1(x)


        # x = self.fc2(x)
        # x = self.bn2(x)
        # x = self.act2(x)
        # x = self.drop2(x)

        
        # x = self.fc3(x)
        # x = self.bn3(x)
        # x = self.act3(x)
        # x = self.drop3(x)

        # x = self.output(x)

        x = self.drop1(self.act1(self.bn1(self.fc1(X))))
        x = self.drop2(self.act2(self.bn2(self.fc2(x))))
        x = self.drop3(self.act3(self.bn3(self.fc3(x))))
        x = self.output(x)

        # Nota la última capa de salida 'output' no se activa debido a que CrossEntropyLoss usa LogSoftmax internamente. 
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

# --- 1. CONFIGURACIÓN DEL DISPOSITIVO (GPU vs CPU) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Parámetros de la red
input_size = X_tr.shape[1]
output_size = 2   # 2 clases



X_train_t = torch.from_numpy(X_tr).to(torch.float32)
Y_train_t = torch.from_numpy(Y_train).long() 

X_val_t = torch.from_numpy(X_val_vectorized).to(torch.float32)



model = MLP(input_size, output_size)
model.to(device) 

# Calcular pesos de clases para el desbalance
classes = np.unique(Y_train)
weights_calc = compute_class_weight('balanced', classes=classes, y=Y_train)
weights_calc[1] = weights_calc[1] * FACTOR_BALANCE
weights = torch.tensor(weights_calc).float()


weights = weights.to(device) 

criterion = nn.CrossEntropyLoss(weight=weights) 
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

print("Iniciando entrenamiento en PyTorch...")

for epoch in range(EPOCHS):
    model.train()  
    lossTotal = 0
    
    
    dataloader = create_minibatches(X_train_t, Y_train_t, batch_size=BATCH_SIZE)
    
    for X_tra, y_tr in dataloader:
        # --- 4. MOVER EL BATCH ACTUAL A GPU ---
        X_tra = X_tra.to(device)
        y_tr = y_tr.to(device)
        
        optimizer.zero_grad()
        
        # Propagación hacia adelante
        y_pred = model(X_tra)
        
        loss = criterion(y_pred, y_tr)
        lossTotal += loss.item()
        
        loss.backward()
        optimizer.step()
        
        if np.random.random() < 0.01: 
            print(f"Batch Error : {loss.item():.4f}")

    print(f"Época {epoch+1}/{EPOCHS}, Pérdida promedio: {lossTotal/len(dataloader):.4f}")
    
    # --- VALIDACIÓN ---
    model.eval()
    with torch.no_grad():
        # Mover datos de validación a GPU
        X_val_gpu = X_val_t.to(device)
        
        Y_val_tensor = torch.from_numpy(Y_val).long().to(device)
        y_pred_logits = model(X_val_gpu)
        
        val_loss = criterion(y_pred_logits, Y_val_tensor)
        # Softmax y Argmax
        y_pred_prob = torch.softmax(y_pred_logits, dim=1)
        y_pred_class = torch.argmax(y_pred_prob, dim=1)
        

        y_pred_numpy = y_pred_class.cpu().numpy()
        
        print(f"--- Resultados Val Época {epoch+1} ---")
        # Usamos 'binary' si solo hay 2 clases y el humor es la clase positiva (1)
        # Usamos 'macro' si te importa el promedio de ambas
        print("P =", precision_score(Y_val, y_pred_numpy, average='macro'))
        print("R =", recall_score(Y_val, y_pred_numpy, average='macro'))
        print("F1=", f1_score(Y_val, y_pred_numpy, average='macro'))
        print("Acc=", accuracy_score(Y_val, y_pred_numpy))
        print("--------------------------------------")
    scheduler.step(val_loss)

Usando dispositivo: cuda
Iniciando entrenamiento en PyTorch...
Batch Error : 2.3665
Batch Error : 2.0409
Batch Error : 1.9973
Época 1/50, Pérdida promedio: 2.1793
--- Resultados Val Época 1 ---
P = 0.7485119047619048
R = 0.7447675831112917
F1= 0.7464989633059456
Acc= 0.7663461538461539
--------------------------------------
Época 2/50, Pérdida promedio: 1.1198
--- Resultados Val Época 2 ---
P = 0.7620573467230444
R = 0.7527650659752507
F1= 0.7566822740849
Acc= 0.7778846153846154
--------------------------------------
Época 3/50, Pérdida promedio: 0.8125
--- Resultados Val Época 3 ---
P = 0.7622606683737798
R = 0.7410775094691313
F1= 0.7484276729559749
Acc= 0.775
--------------------------------------
Batch Error : 0.5315
Época 4/50, Pérdida promedio: 0.6590
--- Resultados Val Época 4 ---
P = 0.7800982586194839
R = 0.7552264426734215
F1= 0.7637224298745475
Acc= 0.7894230769230769
--------------------------------------
Batch Error : 0.2484
Batch Error : 0.2263
Batch Error : 0.6830
Batch 

In [ ]:
X_val_t = torch.from_numpy(X_val_vectorized).to(torch.float32)


device = next(model.parameters()).device # Truco para detectar dónde está el modelo automáticamente
X_val_t = X_val_t.to(device)

# 2. INFERENCIA
model.eval() # Modo evaluación (apaga Dropout y Batch Norm)

with torch.no_grad(): # Ahorra memoria y cálculo
    # Predicción (Logits)
    y_pred_logits = model(X_val_t)
    
    # Obtener la clase con mayor probabilidad (Argmax)
    y_pred_class_tensor = torch.argmax(y_pred_logits, dim=1)

# 3. POST-PROCESAMIENTO

y_pred_val_numpy = y_pred_class_tensor.cpu().numpy()

print(y_pred_val_numpy)


[1 0 0 ... 1 0 1]


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print("--- Matriz de Confusión ---")
cm = confusion_matrix(Y_val, y_pred_val_numpy)
print(cm)

print("\n--- Reporte de Clasificación ---")

print(classification_report(Y_val, y_pred_val_numpy, digits=4, zero_division='warn', target_names=['No Humor', 'Humor']))


--- Matriz de Confusión ---
[[556 103]
 [ 99 282]]

--- Reporte de Clasificación ---
              precision    recall  f1-score   support

    No Humor     0.8489    0.8437    0.8463       659
       Humor     0.7325    0.7402    0.7363       381

    accuracy                         0.8058      1040
   macro avg     0.7907    0.7919    0.7913      1040
weighted avg     0.8062    0.8058    0.8060      1040



In [ ]:
# 1. PREPARAR DATOS
# Convertir a tensor float
X_testing = torch.from_numpy(X_t).to(torch.float32)


device = next(model.parameters()).device 
X_testing = X_testing.to(device)

# 2. INFERENCIA
model.eval() # Apaga Dropout y Batch Norm

with torch.no_grad(): 
    # Predicción de logits
    y_pred_test_logits = model(X_testing)

# 3. PROCESAR RESULTADOS
# Obtener la clase (0 o 1) usando argmax
y_pred_test_indices = torch.argmax(y_pred_test_logits, dim=1)


y_pred_final = y_pred_test_indices.cpu().numpy()

print("Predicciones generadas:")
print(y_pred_final)


unique, counts = np.unique(y_pred_final, return_counts=True)
print("\nConteo de clases predichas:")
print(dict(zip(unique, counts)))

Predicciones generadas:
[1 0 0 ... 0 0 1]

Conteo de clases predichas:
{np.int64(0): np.int64(3531), np.int64(1): np.int64(2069)}


In [40]:
def guardar_resultados(datos, archivo):

    df = pd.DataFrame(datos, columns=['klass'])

    df['id'] = df.index + 1

    df = df[['id', 'klass']]

    df.to_csv(archivo, index=False)

    print(f" {datos} guardado exitosamente!")

In [54]:
guardar_resultados(y_pred_final, f" neuronas{1024}, {256}, {64}, {LEARNING_RATE}, {NGRAM_RANGE}, TFIDF {USE_TFIDF}, {BATCH_SIZE}, preprocesador {preproc}, {MAX_FEATURES}, balance {FACTOR_BALANCE}.csv")

 [1 0 0 ... 0 0 1] guardado exitosamente!
